In [2]:
import (
    "fmt"
    "time"
    "bytes"
)

# Map

If you would like to call a pointer receiver method on a value, the value must be addressable, __not every__ variable is addressable though. __Map elements__ are not addressable and __variables referenced through interfaces__ are also not addressable. You have to call its pointer instead

In [6]:
type data struct {  
    name string
}

func (p *data) print() {  
    fmt.Println("name:",p.name)
}

type printer interface {  
    print()
}

In [7]:
    d1 := data{"one"}
    d1.print() //ok

//     var in printer = data{"two"} //error
    var in printer = &data{"two"} //ok
    in.print()

    m := map[string]data{"x":data{"three"}} //error
//     m := map[string]*data{"x":&data{"three"}} //ok
    m["x"].print() 

    n := map[string]*data {"x":&data{"one"}}
    n["z"] = &data{}
    n["z"].name = "whatthehell?" //???
    n["z"].print()
    

name: one
name: two


ERROR: reflect.Value.Addr of unaddressable value

# Deferred Calls

The deferred calls are executed at the end of the containing function __in reverse order__

In [7]:
func DeferFuncPrint() {
    defer func() {
        fmt.Println("first defer func")
    }()
    
    defer func() {
        fmt.Println("second defer func")
    }()
    
    for _, i := range []int{1, 2, 3} {
        i := i
        defer func(index int){
            fmt.Printf("defer func in for loop: %d\n", index)
        }(i)
        fmt.Printf("func for loop: %d\n", i)
    }
    
    fmt.Println("main func")
}

In [14]:
DeferFuncPrint()

func for loop: 1
func for loop: 2
func for loop: 3
main func
defer func in for loop: 3
defer func in for loop: 2
defer func in for loop: 1
second defer func
first defer func


# Slice Data "Corruption"

For path rewriting scenario, it does work as expected if you reslice the slice twice, because those newly created slices referenced the same underlying array. If you modify the subslice, it actually change the referenced arrary. 

The solution can be allocating new slices and copying the data you need or use full slice expression sli := 

```
sli := a[low : high : max]
```
The syntax has been introduced in Go 1.2 "Re-slicing slices in Golang" which is documented in Full slice expressions.
That slice has:

* indices starting at 0
* length equals to high - low
* capacity equals to max - low

In [8]:
path := []byte("AAAA/BBBBBBBBB")
    sepIndex := bytes.IndexByte(path,'/')
    fmt.Printf("sep index: %d\n", sepIndex)
    dir1 := path[:sepIndex:sepIndex] //full slice expression
//     dir1 := path[:sepIndex] 
    dir2 := path[sepIndex+1:]
    fmt.Println("dir1 =>",string(dir1)) //prints: dir1 => AAAA
    fmt.Println("dir2 =>",string(dir2)) //prints: dir2 => BBBBBBBBB

    dir1 = append(dir1,"suffix"...)
    path = bytes.Join([][]byte{dir1,dir2},[]byte{'/'})

    fmt.Println("dir1 =>",string(dir1)) //prints: dir1 => AAAAsuffix
    fmt.Println("dir2 =>",string(dir2)) //prints: dir2 => BBBBBBBBB (ok now)

    fmt.Println("new path =>",string(path))

sep index: 4
dir1 => AAAA
dir2 => BBBBBBBBB
dir1 => AAAAsuffix
dir2 => BBBBBBBBB
new path => AAAAsuffix/BBBBBBBBB


33 <nil>

# Type ssertion

Failed type assertions return the "zero value" for the target type used in the assertion statement.

In [18]:
    var data interface{} = "great"

    // zero value for type int which is 0
    if data, ok := data.(int); ok {
        fmt.Println("[is an int] value =>",data)
    } else {
        fmt.Println("[not an int] value =>",data) 
        //prints: [not an int] value => 0 (not "great")
    }

    // work as expected
    if res, ok := data.(int); ok {
        fmt.Println("[is an int] value =>",res)
    } else {
        fmt.Println("[not an int] value =>",data) 
        //prints: [not an int] value => great (as expected)
    }

[not an int] value => 0
[not an int] value => great


In [9]:
type TestStruct struct {}

var a *TestStruct = nil
var b interface{}

fmt.Printf("b is %+v\n", b)

b = a

if b == nil {
    fmt.Printf("b is nil\n")
} else {
    fmt.Printf("b is not nil\n")
}


b is <nil>
b is not nil


In [10]:
type GenericNum interface {
    int64 | float64
}

ERROR: repl.go:2:11: expected ';', found '|' (and 1 more errors)